<a href="https://colab.research.google.com/github/DataGuy-Kariuki/Data-Science-and-ML-BOOKS-/blob/main/Project_Introduction_to_Computer_Vision_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement

## Business Context

- Workplace safety in hazardous environments like construction sites and industrial plants is crucial to prevent accidents and injuries. One of the most important safety measures is ensuring workers wear safety helmets, which protect against head injuries from falling objects and machinery.
- Non-compliance with helmet regulations increases the risk of serious injuries or fatalities, making effective monitoring essential, especially in large-scale operations where manual oversight is prone to errors and inefficiency.
- To overcome these challenges, SafeGuard Corp plans to develop an automated image analysis system capable of detecting whether workers are wearing safety helmets. This system will improve safety enforcement, ensuring compliance and reducing the risk of head injuries.
- By automating helmet monitoring, SafeGuard aims to enhance efficiency, scalability, and accuracy, ultimately fostering a safer work environment while minimizing human error in safety oversight.

## Objective

- As a data scientist at SafeGuard Corp, you are tasked with developing an image classification model that classifies images into one of two categories:
- With Helmet: Workers wearing safety helmets.
- Without Helmet: Workers not wearing safety helmets.


## Data Description
The dataset consists of 631 images, equally divided into two categories:

- With Helmet: 311 images showing workers wearing helmets.
- Without Helmet: 320 images showing workers not wearing helmets.

## Dataset Characteristics:

- Variations in Conditions: Images include diverse environments such as construction sites, factories, and industrial settings, with variations in lighting, angles, and worker postures to simulate real-world conditions.
- Worker Activities: Workers are depicted in different actions such as standing, using tools, or moving, ensuring robust model learning for various scenarios.

## Installing and Importing the Necessary Libraries

In [ ]:
!pip install tensorflow[and-cuda] numpy==1.25.2 -q

## Note

- After running the above cell, kindly restart the notebook kernel (for Jupyter Notebook) or runtime (for Google Colab) and run all cells sequentially from the next cell.

- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in this notebook.

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import math
import cv2


# Tensorflow modules
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16

# Display images using OpenCV
from google.colab.patches import cv2_imshow

#Imports functions for evaluating the performance of machine learning models
from sklearn.metrics import confusion_matrix, f1_score,accuracy_score, recall_score, precision_score, classification_report
from sklearn.metrics import mean_squared_error as mse

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set the seed using keras.utils.set_random_seed. This will set:
# 1) `numpy` seed
# 2) backend random seed
# 3) `python` random seed
tf.keras.utils.set_random_seed(812)

## Data Overview

  ## Loading the data

In [ ]:
# Uncomment and run the following code in case Google Colab is being used
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
images = np.load('/content/drive/MyDrive/task 7/images_proj.npy') #Complete the code to load the images

labels = pd.read_csv('/content/drive/MyDrive/task 7/Labels_proj.csv') #Complete the code to load the labels

In [ ]:
print(images.shape) #Complete the code to print the shape of the images
print(labels.shape) #Complete the code to print the shape of the labels

## Exploratory Data Analysis

- Plot random images from each of the classes and print their corresponding labels.

In [ ]:
# Create two lists of indices, one for each class
helmet_indices = np.where(labels == 1)[0]
no_helmet_indices = np.where(labels == 0)[0]

# Select one image from each class
helmet_img = images[np.random.choice(helmet_indices)]
no_helmet_img = images[np.random.choice(no_helmet_indices)]

# Plot the images
fig, axes = plt.subplots(1, 2, figsize=(8, 4))

# Display "With Helmet" image
axes[0].imshow(helmet_img)
axes[0].set_title("Worker WITH Helmet")
axes[0].axis('off')

# Display "Without Helmet" image
axes[1].imshow(no_helmet_img)
axes[1].set_title("Worker WITHOUT Helmet")
axes[1].axis('off')

# Show the plots
plt.tight_layout()
plt.show()

## Checking for class imbalance

In [ ]:
# Create a count plot
plt.figure(figsize=(6, 4))
ax = sns.countplot(x=labels.iloc[:, 0], palette=['red', 'green'])

# Add exact counts on top of bars
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2, p.get_height()),
                ha='center', va='bottom', fontsize=10, )

# Add labels
plt.xlabel("Class Labels", fontsize=12)
plt.ylabel("Number of Images", fontsize=12)
plt.title("Count of Images per Class", fontsize=14)
plt.xticks(ticks=[0, 1], labels=["Without Helmet (0)", "With Helmet (1)"])  # Rename x-axis labels

# Show plot
plt.show()

- The dataset is slightly imbalanced as we have 320 workers working without helmet and 311 workers using helment.
- The difference between the two classes (311 vs. 320) is very small. This is not a major issue and is a good thing. A dataset is considered heavily imbalanced when one class has a significantly larger number of samples than the other (e.g., 90% of the data belongs to one class).

## Data Preprocessing

### Converting images to grayscale

In [ ]:
images_gray = []
for i in range(len(images)):
    img_gray = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    images_gray.append(img_gray)

# Display a sample grayscale image
n = 45 #Complete the code to define an index value
cv2_imshow(images_gray[n])

## Splitting the dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(np.array(images),labels , test_size=0.3, random_state=42,stratify=labels) #Complete the code to define the test_size
X_val, X_test, y_val, y_test = train_test_split(X_temp,y_temp , test_size=0.5, random_state=42,stratify=y_temp) #Complete the code to define the test_size

In [ ]:
print(X_train.shape,y_train.shape) #Complete the code to print the shape of the train data
print(X_val.shape,y_val.shape) #Complete the code to print the shape of the validation data
print(X_test.shape,y_test.shape) #Complete the code to print the shape of the test data

## Data Normalization

- Since the image pixel values range from 0-255, our method of normalization here will be scaling - we shall divide all the pixel values by 255 to standardize the images to have values between 0-1.

In [ ]:
X_train_normalized = X_train.astype('float32')/255 #Complete the code to normalize the training images
X_val_normalized = X_val.astype('float32')/255  #Complete the code to normalize the validation images
X_test_normalized = X_test.astype('float32')/255   #Complete the code to normalize the test images

## Model Building

### Utility Functions

In [ ]:
# defining a function to compute different metrics to check performance of a classification model built using statsmodels
def model_performance_classification(model, predictors, target):
    """
    Function to compute different metrics to check classification model performance

    model: classifier
    predictors: independent variables
    target: dependent variable
    """

    # checking which probabilities are greater than threshold
    pred = model.predict(predictors).reshape(-1)>0.5

    target = target.to_numpy().reshape(-1)


    acc = accuracy_score(target, pred)  # to compute Accuracy
    recall = recall_score(target, pred, average='weighted')  # to compute Recall
    precision = precision_score(target, pred, average='weighted')  # to compute Precision
    f1 = f1_score(target, pred, average='weighted')  # to compute F1-score

    # creating a dataframe of metrics
    df_perf = pd.DataFrame({"Accuracy": acc, "Recall": recall, "Precision": precision, "F1 Score": f1,},index=[0],)

    return df_perf

In [ ]:
def plot_confusion_matrix(model,predictors,target,ml=False):
    """
    Function to plot the confusion matrix

    model: classifier
    predictors: independent variables
    target: dependent variable
    ml: To specify if the model used is an sklearn ML model or not (True means ML model)
    """

    # checking which probabilities are greater than threshold
    pred = model.predict(predictors).reshape(-1)>0.5

    target = target.to_numpy().reshape(-1)

    # Plotting the Confusion Matrix using confusion matrix() function which is also predefined tensorflow module
    confusion_matrix = tf.math.confusion_matrix(target,pred)
    f, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(
        confusion_matrix,
        annot=True,
        linewidths=.4,
        fmt="d",
        square=True,
        ax=ax
    )
    plt.show()

## Model 1: Simple Convolutional Neural Network (CNN)
- Let's build a CNN Model.

#### The model has 2 main parts:

- The Feature Extraction layers which are comprised of convolutional and pooling layers.
- The Fully Connected classification layers for prediction.


In [ ]:
# Initializing Model
model_1 = Sequential()

# Convolutional layers
model_1.add(Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=(200,200,3))) #Complete the code to define the shape of the input image
model_1.add(MaxPooling2D((4, 4), padding='same'))
model_1.add(Conv2D(64, (3, 3), activation='relu', padding="same")) #Complete the code to define the number of output channels,the kernel shape and the activation function
model_1.add(MaxPooling2D((2,2), padding='same')) #Complete the code to define the shape of the pooling kernel
model_1.add(Conv2D(128, (3,3), activation='relu', padding="same")) #Complete the code to define the number of output channels,the kernel shape and the activation function

# Flatten and Dense layers
model_1.add(Flatten())
model_1.add(Dense(64, activation='relu'))
model_1.add(Dense(1, activation='sigmoid'))  #Complete the code to define the number of neurons in the output layer and the activation function

# Compile with Adam Optimizer
opt = Adam(learning_rate=0.001) #Complete the code to define the learning rate.
model_1.compile(optimizer=opt, loss='binary_crossentropy', metrics=["accuracy", "precision", "recall"]) #Complete the code to define the metric of choice from Precision,f1_score,Recall

# Summary
model_1.summary()

In [ ]:
history_1 = model_1.fit(
            X_train_normalized, y_train,
            epochs=20, #Complete the code to define the number of epochs
            validation_data=(X_val_normalized,y_val),
            shuffle=True,
            batch_size=32, #Complete the code to define the batch size
            verbose=2
)

In [ ]:
plt.plot(history_1.history['accuracy']) #Complete the code to plot the train metrics
plt.plot(history_1.history['val_accuracy']) #Complete the code to plot the validation data metrics
plt.title('Model Accuracy') #Complete the code to define the title for the plot
plt.ylabel('Accuracy') #Complete the code to define the label for the y-axis
plt.xlabel('Epoch') #Complete the code to define the label for the x-axis
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
model_1_train_perf = model_performance_classification(model_1, X_train_normalized,y_train)

print("Train performance metrics")
print(model_1_train_perf)

- Accuracy (1.0): The model correctly classified every single image in the training set.
- Recall (1.0): The model correctly identified all the positive cases (workers with helmets) in the training set. There were no false negatives.
- Precision (1.0): When the model predicted a positive case (worker with a helmet), it was always correct. There were no false positives.
- F1 Score (1.0): This is the harmonic mean of precision and recall, and a score of 1.0 indicates a perfect balance between the two.

In [ ]:
plot_confusion_matrix(model_1,X_train_normalized,y_train)

In [ ]:
model_1_valid_perf = model_performance_classification(model_1, X_val_normalized,y_val)

print("Validation performance metrics")
print(model_1_valid_perf)

- Accuracy (0.989474): This means that approximately 98.94% of the images in the validation set were correctly classified by the model. This is a very high accuracy, suggesting the model is generally good at distinguishing between images with and without helmets.
- Recall (0.989474): This indicates that the model correctly identified about 98.94% of the actual positive cases (workers with helmets) in the validation set. This is important for minimizing false negatives (missing a worker who is not wearing a helmet).
- Precision (0.989683): This means that when the model predicted a positive case (worker with a helmet), it was correct about 98.96% of the time. This is important for minimizing false positives (incorrectly flagging a worker as not wearing a helmet when they are).
-  The F1 score is a single metric that balances both precision and recall. An F1 score of 0.989474 means that your model has a very good balance between correctly identifying positive cases (recall) and not incorrectly labeling negative cases as positive (precision) on the validation data.

In [ ]:
plot_confusion_matrix(model_1,X_val_normalized,y_val)

## Vizualizing the predictions

In [ ]:
# For index 12 (corrected)
plt.figure(figsize=(2,2))
plt.imshow(X_val[12])
plt.show()
prediction = model_1.predict(X_val_normalized[12].reshape(1,200,200,3))  # Changed to 12
predicted_label = prediction[0][0]>0.5
print('Predicted Label:', 1 if predicted_label else 0)
true_label = y_val.iloc[12]
print('True Label:', true_label)

# For index 33 (corrected)
plt.figure(figsize=(2,2))
plt.imshow(X_val[33])
plt.show()
prediction = model_1.predict(X_val_normalized[33].reshape(1,200,200,3))  # Changed to 33
predicted_label = prediction[0][0]>0.5
print('Predicted Label:', 1 if predicted_label else 0)
true_label = y_val.iloc[33]
print('True Label:', true_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step:
- Processed 1 image in 30 milliseconds

Predicted Label: 1:
- Your model predicted "With Helmet" ✅

True Label: Label 1:
- The actual label is "With Helmet" ✅

This image is from row 173 in your validation set

## Model 2: (VGG-16 (Base))

- We will be loading a pre-built architecture - VGG16, which was trained on the ImageNet dataset and is the runner-up in the ImageNet competition in 2014.

- or training VGG16, we will directly use the convolutional and pooling layers and freeze their weights i.e. no training will be done on them. For classification, we will add a Flatten and a single dense layer.

In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3)) #Complete the code to define the shape of the image
vgg_model.summary()

In [ ]:
# Making all the layers of the VGG model non-trainable. i.e. freezing them
for layer in vgg_model.layers:
    layer.trainable = False

In [ ]:
model_2 = Sequential()

# Adding the convolutional part of the VGG16 model from above
model_2.add(vgg_model)

# Flattening the output of the VGG16 model because it is from a convolutional layer
model_2.add(Flatten())

# Adding a dense output layer
model_2.add(Dense(1, activation='sigmoid')) #Complete the code to define the number of neurons in the output layer.

In [ ]:
opt = Adam(learning_rate=0.001) #Complete the code to define the learning rate
# Compile model
model_2.compile(optimizer=opt, loss=keras.losses.BinaryCrossentropy(), metrics=["accuracy", "precision", "recall"]) #Complete the code to define the metrics

In [ ]:
# Generating the summary of the model
model_2.summary()

In [ ]:
train_datagen = ImageDataGenerator()

In [ ]:
# Epochs
epochs = 10 #Complete the code to define the epochs
# Batch size
batch_size = 32 #Complete the code to define the batch size

history_2 = model_2.fit(train_datagen.flow(X_train_normalized,y_train,
                                      batch_size=batch_size,
                                      seed=42,
                                      shuffle=False),
                    epochs=epochs,
                    steps_per_epoch=X_train_normalized.shape[0] // batch_size,
                    validation_data=(X_val_normalized,y_val),
                    verbose=1)

In [ ]:
plt.plot(history_2.history['accuracy']) #Complete the code to plot the train metrics
plt.plot(history_2.history['val_accuracy']) #Complete the code to plot the validation data metrics
plt.title('Model 2 Accuracy') #Complete the code to define the title for the plot
plt.ylabel('Accuracy') #Complete the code to define the label for the y-axis
plt.xlabel('Epoch') #Complete the code to define the label for the x-axis
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
model_2_train_perf = model_performance_classification(model_2,X_train_normalized,y_train)

print("Train performance metrics")
print(model_2_train_perf)

In [ ]:
plot_confusion_matrix(model_2,X_train_normalized,y_train)

In [ ]:
model_2_valid_perf = model_performance_classification(model_2, X_val_normalized,y_val)

print("Validation performance metrics")
print(model_2_valid_perf)

In [ ]:
plot_confusion_matrix(model_2,X_val_normalized,y_val)

## Visualizing the prediction:

In [ ]:
# For first prediction (using index 0 as example)
plt.figure(figsize=(2,2))
plt.imshow(X_val[20]) #Complete the code to define the index
plt.show()
prediction = model_2.predict(X_val_normalized[20].reshape(1,200,200,3)) #Complete the code to define the index
predicted_label = prediction[0][0]>0.5  # Extract the predicted class label
print('Predicted Label:', 1 if predicted_label else 0)
# Fix indexing issue in y_val
true_label = y_val.iloc[20] #Complete the code to define the index
print('True Label:', true_label)

# For second prediction (using index 1 as example)
plt.figure(figsize=(2,2))
plt.imshow(X_val[11]) #Complete the code to define the index
plt.show()
prediction = model_2.predict(X_val_normalized[11].reshape(1,200,200,3)) #Complete the code to define the index
predicted_label = prediction[0][0]>0.5  # Extract the predicted class label
print('Predicted Label:', 1 if predicted_label else 0)
# Fix indexing issue in y_val
true_label = y_val.iloc[11] #Complete the code to define the index
print('True Label:', true_label)

## Model 3: (VGG-16 (Base + FFNN))

- We will directly use the convolutional and pooling layers and freeze their weights i.e. no training will be done on them. For classification, we will add a Flatten layer and a Feed Forward Neural Network.

In [ ]:
model_3 = Sequential()

# Adding the convolutional part of the VGG16 model from above
model_3.add(vgg_model)

# Flattening the output of the VGG16 model because it is from a convolutional layer
model_3.add(Flatten())

# Adding the Feed Forward neural network
model_3.add(Dense(256, activation='relu')) #Complete the code to define the number of neurons and the activation function
model_3.add(Dropout(rate=0.5)) #Complete the code to define the dropout rate
model_3.add(Dense(128, activation='relu')) #Complete the code to define the number of neurons and the activation function

# Adding a dense output layer
model_3.add(Dense(1, activation='sigmoid')) #Complete the code to define the number of neurons in the output layer and the activation function

In [ ]:
opt = Adam(learning_rate=0.001) #Complete the code to define the learning rate

In [ ]:
# Compile model
model_3.compile(optimizer=opt, loss=keras.losses.BinaryCrossentropy(), metrics=["accuracy", "precision", "recall"]) #Complete the code to define the metrics

In [ ]:
# Generating the summary of the model
model_3.summary()

In [ ]:
history_3 = model_3.fit(train_datagen.flow(X_train_normalized,y_train,
                                       batch_size=32, #Complete the code to define the batch size
                                       seed=42,
                                       shuffle=False),
                    epochs=15, #Complete the code to define the number of epochs
                    steps_per_epoch=X_train_normalized.shape[0] // batch_size,
                    validation_data=(X_val_normalized,y_val),
                    verbose=1)

In [ ]:
plt.plot(history_3.history['accuracy']) #Complete the code to plot the train metrics
plt.plot(history_3.history['val_accuracy']) #Complete the code to plot the validation data metrics
plt.title('Model 3 Accuracy') #Complete the code to define the title for the plot
plt.ylabel('Accuracy') #Complete the code to define the label for the y-axis
plt.xlabel('Epoch') #Complete the code to define the label for the x-axis
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
model_3_train_perf = model_performance_classification(model_3, X_train_normalized,y_train)

print("Train performance metrics")
print(model_3_train_perf)

In [ ]:
plot_confusion_matrix(model_3,X_train_normalized,y_train)

In [ ]:
model_3_valid_perf = model_performance_classification(model_3, X_val_normalized,y_val)

print("Validation performance metrics")
print(model_3_valid_perf)

In [ ]:
plot_confusion_matrix(model_3,X_val_normalized,y_val)

## Visualizing the predictions

In [ ]:
# For first prediction (using index 0 as example)
plt.figure(figsize=(2,2))
plt.imshow(X_val[0]) #Complete the code to define the index
plt.show()
prediction = model_3.predict(X_val_normalized[0].reshape(1,200,200,3)) #Complete the code to define the index
predicted_label = prediction[0][0]>0.5  # Extract the predicted class label
print('Predicted Label:', 1 if predicted_label else 0)
# Fix indexing issue in y_val
true_label = y_val.iloc[0] #Complete the code to define the index
print('True Label:', true_label)

# For second prediction (using index 1 as example)
plt.figure(figsize=(2,2))
plt.imshow(X_val[15]) #Complete the code to define the index
plt.show()
prediction = model_3.predict(X_val_normalized[15].reshape(1,200,200,3)) #Complete the code to define the index
predicted_label = prediction[0][0]>0.5  # Extract the predicted class label
print('Predicted Label:', 1 if predicted_label else 0)
# Fix indexing issue in y_val
true_label = y_val.iloc[15] #Complete the code to define the index
print('True Label:', true_label)

## Model 4: (VGG-16 (Base + FFNN + Data Augmentation)

In most of the real-world case studies, it is challenging to acquire a large number of images and then train CNNs.

To overcome this problem, one approach we might consider is Data Augmentation.

CNNs have the property of translational invariance, which means they can recognise an object even if its appearance shifts translationally in some way. - Taking this attribute into account, we can augment the images using the techniques listed below

- Horizontal Flip (should be set to True/False)
- Vertical Flip (should be set to True/False)
- Height Shift (should be between 0 and 1)
- Width Shift (should be between 0 and 1)
- Rotation (should be between 0 and 180)
- Shear (should be between 0 and 1)
- Zoom (should be between 0 and 1) etc.

Remember, data augmentation should not be used in the validation/test data set.

In [ ]:
model_4 = Sequential()

# Adding the convolutional part of the VGG16 model from above
model_4.add(vgg_model)

# Flattening the output of the VGG16 model because it is from a convolutional layer
model_4.add(Flatten())

# Adding the Feed Forward neural network
model_4.add(Dense(256, activation='relu')) #Complete the code to define the number of neurons and the activation function
model_4.add(Dropout(rate=0.5)) #Complete the code to define the dropout rate
model_4.add(Dense(128, activation='relu')) #Complete the code to define the number of neurons and the activation function

# Adding a dense output layer
model_4.add(Dense(1, activation='sigmoid')) #Complete the code to define the number of neurons in the output layer and the activation function

In [ ]:
opt = Adam(learning_rate=0.001)
# Compile model
model_4.compile(optimizer=opt, loss=keras.losses.BinaryCrossentropy(), metrics=["accuracy"]) #Complete the code to define the metrics

In [ ]:
# Generating the summary of the model
model_4.summary()

In [ ]:
# Applying data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,           #Complete the code to define the range for rotation
    fill_mode='nearest',
    width_shift_range=0.2,       #Complete the code to define the parameters for the data augmentation
    height_shift_range=0.2,      #Complete the code to define the parameters for the data augmentation
    shear_range=0.2,             #Complete the code to define the parameters for the data augmentation
    zoom_range=0.2               #Complete the code to define the parameters for the data augmentation
)

In [ ]:
history_4 = model_4.fit(train_datagen.flow(X_train_normalized,y_train,
                                       batch_size=32, #Complete the code to define the batch size
                                       seed=42,
                                       shuffle=False),
                    epochs=15, #Assuming epochs is defined earlier, otherwise use a number like 15
                    steps_per_epoch=X_train_normalized.shape[0] // batch_size,
                    validation_data=(X_val_normalized,y_val),
                    verbose=1)

In [ ]:
plt.plot(history_4.history['accuracy']) #Complete the code to plot the train metrics
plt.plot(history_4.history['val_accuracy']) #Complete the code to plot the validation data metrics
plt.title('Model 4 Accuracy (With Data Augmentation)') #Complete the code to define the title for the plot
plt.ylabel('Accuracy') #Complete the code to define the label for the y-axis
plt.xlabel('Epoch') #Complete the code to define the label for the x-axis
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
model_4_train_perf = model_performance_classification(model_4, X_train_normalized,y_train)

print("Train performance metrics")
print(model_4_train_perf)

In [ ]:
plot_confusion_matrix(model_4,X_train_normalized,y_train)

In [ ]:
model_4_valid_perf = model_performance_classification(model_4, X_val_normalized,y_val)

print("Validation performance metrics")
print(model_4_valid_perf)

In [ ]:
plot_confusion_matrix(model_4,X_val_normalized,y_val)

## Visualizing the predictions

In [ ]:
# For first prediction (using index 0 as example)
# For first prediction (using index 10)
plt.figure(figsize=(2,2))
plt.imshow(X_val[10]) #Complete the code to define the index
plt.show()
prediction = model_4.predict(X_val_normalized[10].reshape(1,200,200,3)) #Complete the code to define the index
predicted_label = prediction[0][0]>0.5  # Extract the predicted class label (FIXED: use [0][0] not [12][0])
print('Predicted Label:', 1 if predicted_label else 0)
# Fix indexing issue in y_val
true_label = y_val.iloc[10] #Complete the code to define the index
print('True Label:', true_label)

# For second prediction (using index 7)
plt.figure(figsize=(2,2))
plt.imshow(X_val[7]) #Complete the code to define the index
plt.show()
prediction = model_4.predict(X_val_normalized[7].reshape(1,200,200,3)) #Complete the code to define the index
predicted_label = prediction[0][0]>0.5  # Extract the predicted class label
print('Predicted Label:', 1 if predicted_label else 0)
# Fix indexing issue in y_val
true_label = y_val.iloc[7] #Complete the code to define the index
print('True Label:', true_label)

## Model Performance Comparison and Final Model Selection

In [ ]:
# training performance comparison

models_train_comp_df = pd.concat(
    [
        model_1_train_perf.T,
        model_2_train_perf.T,
        model_3_train_perf.T,
        model_4_train_perf.T,
    ],
    axis=1,
)
models_train_comp_df.columns = [
    "Simple Convolutional Neural Network (CNN)","VGG-16 (Base)","VGG-16 (Base+FFNN)","VGG-16 (Base+FFNN+Data Aug)"
]

In [ ]:
models_valid_comp_df = pd.concat(
    [
        model_1_valid_perf.T,
        model_2_valid_perf.T,
        model_3_valid_perf.T,
        model_4_valid_perf.T

    ],
    axis=1,
)
models_valid_comp_df.columns = [
 "Simple Convolutional Neural Network (CNN)","VGG-16 (Base)","VGG-16 (Base+FFNN)","VGG-16 (Base+FFNN+Data Aug)"
]

In [ ]:
models_train_comp_df

In [ ]:
models_valid_comp_df

In [ ]:
models_train_comp_df - models_valid_comp_df

## Test Performance

In [ ]:
model_test_perf = model_performance_classification(model_4, X_test_normalized, y_test) #Complete the code to pass the best model

### Typical Performance Hierarchy:
- Model 4: VGG-16 + FFNN + Augmentation (Best)
- Model 3: VGG-16 + FFNN (Very Good)
- Model 2: VGG-16 Base (Good)
- Model 1: Simple CNN (Baseline)

The data augmentation in Model 4 is what usually makes it outperform Model 3, as it effectively gives you a much larger and more diverse training set without collecting more real images.

In [ ]:
model_test_perf

In [ ]:
plot_confusion_matrix(model_4, X_test_normalized, y_test) #Complete the code to pass the best model

## Actionable Insights & Recommendations